In [2]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky,GRU_NCL,LambdaLayer
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss,BatchNorm2d
from torch.utils.data import DataLoader
from torch.optim import Adam
from pywt import cwt

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values

In [3]:
## change ##
batch_size = 16
clip = .5
name = 'spec_3d_phase'

In [4]:
def transformFun(x):
    _,_,s1 = spectrogram(x,nperseg=256,noverlap=256//4,mode='phase')
    _,_,s2 = spectrogram(x,nperseg=256,noverlap=256//4)
    return np.stack([np.log(s2+1e-6),s1],0)

Prepare data

In [14]:
train_gen = SequenceGenTransform(train,transformFun)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGenTransform(train,transformFun,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [24]:
class CNN_RNN2seq_transpose(nn.Module):
    def __init__(self,conv,linear):
        super().__init__()
        self.conv = conv 
        self.linear = linear
        self.convert = Linear(1,1)
        
    def forward(self,x):
        n = x.shape[0]
        _,x = self.conv(x)
        x = x.contiguous().view(n*4,-1)
        x = self.linear(x).squeeze(1)
        x = x.view(n,4).mean(1,keepdim=True)
        return self.convert(x).squeeze(1)

In [25]:
convNets = Sequential(BatchNorm2d(2),
                      ConvBatchLeaky(2,4,(5,3),stride=(2,1)),
                      Dropout(0.5),
                      ConvBatchLeaky(4,8,(5,3),stride=(2,1)),
                      Dropout(0.5),
                      ConvBatchLeaky(8,16,(5,5),stride=(2,2)),
                      Dropout(0.5),
                      ConvBatchLeaky(16,32,(3,3),stride=(2,2)),
                      Dropout(0.5),
                      LambdaLayer(lambda x:x.view(-1,32*6,192)),
                      GRU_NCL(192,192,2,batch_first=True,dropout=0.5,bidirectional=True,returnH=True))

In [26]:
linear = Sequential(Dropout(0.5),Linear(192,128),LeakyReLU(0.1,True),Dropout(0.25),Linear(128,1))

In [27]:
model = CNN_RNN2seq_transpose(convNets,linear).to('cuda:0')

In [28]:
opt = Adam(trainable_parameter(model),lr=1e-3)

In [29]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [30]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:3.1499250645820913, val_loss:3.288658618927002
epoch:1, train_loss:2.5850988798416576, val_loss:2.40602970123291
epoch:2, train_loss:2.279357135295868, val_loss:2.373061180114746
epoch:3, train_loss:2.257580471153443, val_loss:2.417478084564209
epoch:4, train_loss:2.244616737159399, val_loss:2.3783562183380127
Training completed in 94.51108837127686s


In [31]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip,lossBest=2.373)

epoch:0, train_loss:2.235746955069212, val_loss:2.4133412837982178
epoch:1, train_loss:2.2379325083815136, val_loss:2.378225088119507
epoch:2, train_loss:2.1955712873202105, val_loss:2.3508412837982178
epoch:3, train_loss:2.1763606999929133, val_loss:2.3603460788726807
epoch:4, train_loss:2.200246194234261, val_loss:2.355538845062256
Training completed in 93.81093525886536s


Save Model and make Submission

In [3]:
submission = pd.read_csv('../Data/sample_submission.csv')

In [34]:
test_gen = SequenceGenTransformTest(submission.seg_id.tolist(),transformFun)
test_gen = DataLoader(test_gen,batch_size,False,num_workers=2)

In [35]:
submission.iloc[:,1] = predict(model,test_gen)

In [36]:
submission.to_csv('../Submission/'+name+'.csv',index=False)

In [39]:
torch.save(model.state_dict(), '../Model/'+name+'.pt') 